**Import the necessary libraries**

In [212]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG19
import cv2

In [213]:
#the size of the images
img_size = 400

**Compute the content cost of the content and the generated image**

In [214]:
def compute_content_cost(content_output, generated_output):
    a_C = content_output[-1]
    a_G = generated_output[-1]
    _, n_H, n_W, n_C = a_G.shape
    a_C_unrolled = tf.reshape(a_C,shape=[1,n_H*n_W,n_C])
    a_G_unrolled = tf.reshape(a_G,shape=[1,n_H*n_W,n_C])
    J_content = tf.reduce_sum(tf.square(tf.subtract(a_C_unrolled,a_G_unrolled)))/(4*n_W*n_H*n_C)
    return J_content

**the style layers**

In [215]:
STYLE_LAYERS = [
    ('block1_conv2', 0.1),
    ('block2_conv2', 0.1),
    ('block3_conv3', 0.2),
    ('block4_conv3', 0.3),
    ('block5_conv3', 0.3)]

**Compute the style Cost of the style and generated image**

In [216]:
def compute_style_cost(style_image_output, generated_image_output, STYLE_LAYERS=STYLE_LAYERS):
    J_style = 0
    a_S = style_image_output[:-1]
    a_G = generated_image_output[:-1]
    for i, weight in zip(range(len(a_S)), STYLE_LAYERS):  
        _, n_H, n_W, n_C = a_G[i].shape
        a_S[i] = tf.transpose(tf.reshape(a_S[i],shape=[n_H*n_W,n_C]))
        a_G[i] = tf.transpose(tf.reshape(a_G[i],shape=[n_H*n_W,n_C]))
        GS = tf.matmul(a_S[i],tf.transpose(a_S[i]))
        GG = tf.matmul(a_G[i],tf.transpose(a_G[i]))
        J_style_layer = tf.reduce_sum(tf.square(tf.subtract(GS,GG)))/(4*(n_C*n_H*n_W)**2)
        J_style += weight[1] * J_style_layer
    return J_style

**Compute the Overall loss**

In [217]:
def total_cost(J_content,J_style, alpha, beta):
    J = alpha*J_content + beta*J_style
    return J

In [218]:
def load_img(path):
    image = cv2.imread(path,-1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image,(img_size, img_size))
    image = tf.constant(np.reshape(image, ((1,) + image.shape)))
    return image

**load the content image**

In [252]:
content_image = load_img('../input/content-style-images/paris.jpg')
print(content_image.shape)
plt.imshow(content_image[0])

**load the style image**

In [253]:
style_image = load_img('../input/content-style-images/style.jpg')
print(style_image.shape)
plt.imshow(style_image[0])

**initialize the generated image by subtracting the content image from some noise generated uniformly**

In [254]:
def initialize_generated_image(noise_percentage):
    generated_image = tf.Variable(tf.image.convert_image_dtype(content_image, tf.float32))
    noise = tf.random.uniform(tf.shape(generated_image), -noise_percentage, noise_percentage)
    generated_image = tf.subtract(generated_image, noise)
    generated_image = tf.clip_by_value(generated_image, clip_value_min=0.0, clip_value_max=1.0)
    return generated_image

In [255]:
generated_image = initialize_generated_image(0.25)
print(generated_image.shape)
plt.imshow(generated_image[0])

**create the model with the style and content layers of the vgg19 model**

In [256]:
def create_model(style_layers,content_layer):
    vgg = VGG19(include_top=False,input_shape=(img_size, img_size, 3),weights='imagenet')
    style_outputs = [vgg.get_layer(name[0]).output for name in style_layers]
    content_outputs = [vgg.get_layer(name[0]).output for name in content_layer]
    outputs = style_outputs + content_outputs
    model = tf.keras.Model(inputs = [vgg.input], outputs=outputs)
    return model

In [257]:
content_layer = [('block5_conv4', 1)]
vgg_model_outputs = create_model(STYLE_LAYERS, content_layer)

In [258]:
def get_model_output(image):
    variable =  tf.Variable(tf.image.convert_image_dtype(image, tf.float32))
    a = vgg_model_outputs(variable)
    return a

In [259]:
a_C = get_model_output(content_image)[-1]

In [260]:
a_S = get_model_output(style_image)[:-1]

**train the model**

In [269]:
epochs = 1001
alpha = 10
beta = 40
generated_image = tf.Variable(generated_image)
def train(epochs,lr=0.01):
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    J = 0
    for i in range(epochs):
        with tf.GradientTape() as tape:
            a_G = vgg_model_outputs(generated_image)
            J_style = compute_style_cost(a_S, a_G)
            J_content = compute_content_cost(a_C, a_G)
            J = total_cost(J_content,J_style,alpha,beta)
        grad = tape.gradient(J, generated_image)
        optimizer.apply_gradients([(grad, generated_image)])
        generated_image.assign(tf.clip_by_value(generated_image, clip_value_min=0.0, clip_value_max=1.0))
        if i % 200 == 0:
            print(f"Epoch {i} Loss {J}")
            tensor = generated_image * 255
            tensor = np.array(tensor, dtype=np.uint8)
            if np.ndim(tensor) > 3:
                assert tensor.shape[0] == 1
                tensor = tensor[0]
                image = Image.fromarray(tensor)
            image.save(f"./image_{i}.jpg")
            plt.imshow(tensor)
            plt.show()

In [270]:
train(epochs)